In [ ]:
from os import listdir
from sys import argv

import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import f1_score, roc_curve, roc_auc_score, precision_recall_curve

from sklearn.decomposition import PCA



In [ ]:
argv = [3, "imgs", "gts"]

path_imgs = sorted([ argv[1] + '/' + i for i in listdir(argv[1]) ])
path_gts = sorted([ argv[2] + '/' + i for i in listdir(argv[2]) ])

names_VI = ["ExG", "ExGR", "CIVE", "VEG", "WI", "NGRDI"]

In [ ]:
GT = np.zeros((512*512*len(path_imgs)), dtype = int)
VI = [ np.zeros((512*512*len(path_imgs)), dtype = float) for i in range(6) ]

In [ ]:
def neighbMax(image):
	O = np.zeros(image.shape, dtype = float)
	ns = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
    

	for lin in range(512):
		for col in range(512):
			_list = []
			for n in ns:
				if lin + n[0] >= 0 and lin + n[0] < 512 and col + n[1] >= 0 and col + n[1] < 512:
					_list.append( image[ lin + n[0] ] [col + n[1] ] )
			O[lin][col] = np.average(sorted(_list, reverse = True)[:3]) # Pega os 3 maiores

	return O

In [ ]:
def neighbMin(image):
	O = np.zeros(image.shape, dtype = float)
	ns = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
    

	for lin in range(512):
		for col in range(512):
			_list = []
			for n in ns:
				if lin + n[0] >= 0 and lin + n[0] < 512 and col + n[1] >= 0 and col + n[1] < 512:
					_list.append( image[ lin + n[0] ] [col + n[1] ] )
			O[lin][col] = np.average(sorted(_list, reverse = False)[:3]) # Pega os 3 menores

	return O

In [ ]:
for i in range(len(path_imgs)):

    
    img = cv2.imread(path_imgs[i], 1)
    gt = cv2.imread(path_gts[i], 0)
    gt = cv2.normalize(gt, gt, 0.0, 1.0, cv2.NORM_MINMAX)
    #GT = np.concatenate((GT, gt.ravel()), axis = 0)
    print(i)
    #print(path_imgs[i])
    
    ims = 512*512
    bls = 16*16
    nbl = 512/16
            

    for lin in range(512)[::16]:
        for col in range(512)[::16]:
            GT[ims*i+ (lin*512+col*16) :ims*i+ (lin*512+col*16) + 16*16 ] = gt[lin:lin+16, col:col+16].ravel()

    #B, G, R = cv2.split(img) # b = img[:,:,0]
    B, G, R = [np.float32(img[:, :, c]) for c in range(3)]
    r = R / (R + G + B)
    g = G / (R + G + B)
    b = B / (R + G + B)

    I = np.zeros(img.shape[:2], dtype = float)

    j = 0

    # ExG
    #print("ExG")
    Z = 2 * g - r - b
    I = cv2.normalize(Z, I, 0.0, 1.0, cv2.NORM_MINMAX)
    #I = neighbMax(I)
    for lin in range(512)[::16]:
        for col in range(512)[::16]:
            VI[j][ims*i+ (lin*512+col*16) :ims*i+ (lin*512+col*16) + 16*16 ] = I[lin:lin+16, col:col+16].ravel()
    j+=1


    # ExGR
    #print("ExGR")
    Z = Z - 1.4 * r - g
    I = cv2.normalize(Z, I, 0.0, 1.0, cv2.NORM_MINMAX)
    #I = neighbMax(I)
    for lin in range(512)[::16]:
        for col in range(512)[::16]:
            VI[j][ims*i+ (lin*512+col*16) :ims*i+ (lin*512+col*16) + 16*16 ] = I[lin:lin+16, col:col+16].ravel()
    j+=1

    # CIVE
    #print("CIVE")
    Z = 0.441 * r - 0.881 * g + 0.385 * b + 18.78745
    I = 1 - cv2.normalize(Z, I, 0.0, 1.0, cv2.NORM_MINMAX)
    #I = neighbMax(I)
    for lin in range(512)[::16]:
        for col in range(512)[::16]:
            VI[j][ims*i+ (lin*512+col*16) :ims*i+ (lin*512+col*16) + 16*16 ] = I[lin:lin+16, col:col+16].ravel()
    j+=1

    # VEG
    #print("VEG")
    Z = g / (2 + r ** 0.667 * b ** (1 - 0.667))
    I = cv2.normalize(Z, I, 0.0, 1.0, cv2.NORM_MINMAX)
    #I = neighbMax(I)
    for lin in range(512)[::16]:
        for col in range(512)[::16]:
            VI[j][ims*i+ (lin*512+col*16) :ims*i+ (lin*512+col*16) + 16*16 ] = I[lin:lin+16, col:col+16].ravel()
    j+=1

    # WI
    #print("WI")
    Z = (g - b) / (r - g + 255)
    I = cv2.normalize(Z, I, 0.0, 1.0, cv2.NORM_MINMAX)
    #I = neighbMax(I)
    for lin in range(512)[::16]:
        for col in range(512)[::16]:
            VI[j][ims*i+ (lin*512+col*16) :ims*i+ (lin*512+col*16) + 16*16 ] = I[lin:lin+16, col:col+16].ravel()
    j+=1

    # NGRDI
    #print("NGRDI")
    Z = (G/2 - R/2) / (G/2 + R/2)
    I = cv2.normalize(Z, I, 0.0, 1.0, cv2.NORM_MINMAX)
    #I = neighbMax(I)
    for lin in range(512)[::16]:
        for col in range(512)[::16]:
            VI[j][ims*i+ (lin*512+col*16) :ims*i+ (lin*512+col*16) + 16*16 ] = I[lin:lin+16, col:col+16].ravel()
    j+=1

# Análise das VIs (curva ROC)

In [ ]:
fpr = [np.array([0]) for i in range(6)]
tpr = [np.array([0]) for i in range(6)]

In [ ]:
for j in range(6):
    print(names_VI[j], "AUC =", roc_auc_score(GT, VI[j]))
    #fpr[j], tpr[j], _ = roc_curve(GT, VI[j])

# Salvar VIs

In [ ]:
VIs_save_dir = "VI_2/MiniNBmax/"

In [ ]:
for j in range(6):
    np.save(VIs_save_dir + names_VI[j], VI[j])

In [ ]:
GT_save_dir = "VI_2/Mini/GT"

In [ ]:
np.save(GT_save_dir , GT)

In [ ]:
X = np.zeros((512*512*len(path_imgs), 6), dtype = float)

# Gerar e salvar PCA das VIs

In [ ]:
for i in range(6):
    X[:, i] = np.load("VI_2/MiniNBmax/" + names_VI[i] + ".npy")

In [ ]:
print(np.count_nonzero(X))

In [ ]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

In [ ]:
print(np.count_nonzero(X_pca))

In [ ]:
np.save(VIs_save_dir + "PCA0", X_pca[:,0])
np.save(VIs_save_dir = "PCA1", X_pca[:,1])